## Imports

In [1]:
import time

import numpy as np
import cv2 as cv
import os
import glob
import matplotlib.pyplot as plt
from cv2 import cvtColor
from numpy.random import uniform
import pdb
from tqdm import tqdm

import utils
from utils import show_image_matplot,show_image_cv

In [2]:
game_tables = utils.extract_game(utils.get_image_paths("antrenare",1))

100%|██████████| 50/50 [00:13<00:00,  3.75it/s]


In [3]:
relevant_game_tables = utils.extract_relevant_game(game_tables)

100%|██████████| 50/50 [00:00<00:00, 134.27it/s]


In [4]:
delimited_tables = utils.draw_lines(relevant_game_tables)

100%|██████████| 50/50 [00:00<00:00, 928.86it/s]


In [5]:
# utils.store_template_numbers1(10)
# utils.store_template_numbers2(8)

In [6]:
# utils.store_binary_templates(100,"./templates/sample1","./templates_binary/sample1")
# utils.store_binary_templates(100)

In [7]:
# new_templates = crop_and_store_templates()

In [8]:
#utils.rewrite_all()

In [25]:
from utils import get_mathable_pieces_numbers, get_lines_coords


def classify_number(patch,templates_path = "./templates_cropped"):

    maxi = -np.inf
    chosen_number = -1
    numbers = get_mathable_pieces_numbers()

    num_whites_patch = np.sum(patch == 255)

    for number in numbers:
        template = cv.imread(templates_path+ "/" + str(number) + ".jpg")
        template = cv.cvtColor(template, cv.COLOR_BGR2GRAY)

        num_whites_template = np.sum(template == 255)

        if num_whites_template * 1.95 < num_whites_patch:
            continue

        corr = cv.matchTemplate(patch,template, cv.TM_CCOEFF_NORMED)
        corr=np.max(corr)

        if corr > maxi:
            maxi = corr
            chosen_number = number


    return chosen_number

def match_numbers(binary_image, offset):
    lines_vertical,lines_horizontal = get_lines_coords()

    coords={}

    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):

            y_min = lines_vertical[j][0][0] + offset
            y_max = lines_vertical[j + 1][1][0] - offset
            x_min = lines_horizontal[i][0][1] + offset
            x_max = lines_horizontal[i + 1][1][1] - offset

            patch = binary_image[x_min:x_max,y_min:y_max].copy()
            # patches.append(patch)

            if (i==6 or i==7) and (j==6 or j==7):
                continue

            mean = np.mean(binary_image[x_min+20:x_max-20,y_min+20:y_max-20].copy())


            if mean > 0:
                predicted_number = classify_number(patch)
                coords[(i,j)] = predicted_number

    return coords


In [54]:
def move_train_data(train_number,dest_path = "./train"):
    img_prefix = str(train_number)+ "_"

    for i in range(9):
        img_name = img_prefix + "0" +str(i+1) + ".jpg"
        img_path = "./antrenare/" + img_name
        img = cv.imread(img_path)
        cv.imwrite(dest_path +"/" + img_name, img)

    for i in range(9,50):
        img_path = "./antrenare/" + img_prefix + str(i+1) + ".jpg"
        img_name = img_prefix +str(i+1) + ".jpg"
        img = cv.imread(img_path)
        cv.imwrite(dest_path +"/" + img_name, img)

# for i in range(4):
#     move_train_data(i+1)



In [58]:

def solve(train_number,src_path = "./train",output_dest_path = "./343_Gabroveanu_Razvan"):

    game_tables = utils.extract_game(utils.get_image_paths(src_path,train_number))
    relevant_game_tables = utils.extract_relevant_game(game_tables)

    visited = np.zeros((14,14),dtype=np.uint8)
    output_file_prefix = "/" + str(train_number) + "_"

    for i in tqdm(range(len(relevant_game_tables))):

        table = relevant_game_tables[i]
        table = cv.cvtColor(table, cv.COLOR_BGR2GRAY)
        _,table_binary = cv.threshold(table,70,255,cv.THRESH_BINARY_INV)

        result_dict = match_numbers(table_binary,8)

        for indexes,predicted_number in result_dict.items():
            row,col = indexes

            if visited[row,col] == 1:
                continue

            output = str(row+1) + chr(65 + col)+ " " + str(predicted_number)
            output_file_path = output_dest_path + output_file_prefix

            if i in range(0,9):
                output_file_path = output_file_path + "0"

            output_file_path = output_file_path + str(i+1) + ".txt"

            with open(output_file_path,"w") as file:
                file.write(output)

            visited[row,col] = 1

    with open(output_dest_path + "/" + str(train_number) + "_scores.txt","w") as file:
        pass

    with open(output_dest_path + "/" + str(train_number) + "_turns.txt","w") as file:
        pass


for i in range(4):
    solve(i+1)



100%|██████████| 50/50 [00:17<00:00,  2.90it/s]


In [41]:

all_sample = utils.extract_game(["./imagini_auxiliare/03.jpg"])
all_sample = utils.extract_relevant_game(all_sample)[0]

all_sample = cv.cvtColor(all_sample, cv.COLOR_BGR2GRAY)
_,all_sample = cv.threshold(all_sample,70,255,cv.THRESH_BINARY_INV)

# show_image_cv("",all_sample,fx=0.3,fy = 0.3)

sample = relevant_game_tables[0]
sample = cv.cvtColor(sample,cv.COLOR_BGR2GRAY)
_,sample = cv.threshold(sample,70,255,cv.THRESH_BINARY_INV)

coords = match_numbers(sample, offset = 8)

100%|██████████| 1/1 [00:00<00:00, 103.06it/s]


In [43]:
print(coords)

{(8, 7): 8}
